Separate the drums from the song

In [6]:
from autostep3.util import highest_index
from autostep3 import better_aubio
import shutil
from os import path
from spleeter.audio import Codec

def separate_drums(audio_path: str, force: bool = False):
	audio_file_name = audio_path[highest_index(audio_path, '/') + 1 : highest_index(audio_path, '.')]
	if force or not path.exists(f'/tmp/{audio_file_name}'):
		from spleeter.separator import Separator
		separator = Separator("spleeter:5stems")
		separator.separate_to_file(audio_path, '/tmp', codec=Codec.MP3, synchronous=True)
	return shutil.copyfile(f'/tmp/{audio_file_name}/drums.mp3', 'separated_drums.wav')

drums_path = separate_drums("test_audio/hyperpop.mp3")

Generate onsets based on the drums
- Change the onset method to experiment

In [7]:
onsets = better_aubio.onset(drums_path, 'complex')

[mp3float @ 0x5631bb28ac80] Could not update timestamps for skipped samples.
[mp3float @ 0x5631bb28ac80] Could not update timestamps for discarded samples.


Load the separated drums for playback

In [13]:
import soundfile as sf
drums, samplerate = sf.read(drums_path, dtype='float32')
num_samples = sf.info(drums_path).frames

Generate the new audio track
- On every onset you will hear a very short clipping sound
- *This can take a while*
- *You can reduce the audio length if needed*

In [9]:
import numpy as np
audible_onsets = np.array([i in onsets for i in range(num_samples // 2)], dtype='float32')

Create 2 audio streams using `sounddevice` to play both tracks simultaneously

In [10]:
import sounddevice as sd
song_stream = sd.OutputStream(channels=2, samplerate=samplerate)
onsets_stream = sd.OutputStream(channels=1, samplerate=samplerate)

The streams need to be multithreaded, since `OutputStream.write()` is a blocking call

In [14]:
from threading import Thread
song_stream.start()
onsets_stream.start()
Thread(target=lambda: song_stream.write(drums)).start()
Thread(target=lambda: onsets_stream.write(audible_onsets)).start()

Run this cell to stop the streams, in case errors occur in the main thread

In [15]:
song_stream.stop()
onsets_stream.stop()